In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("added_features.csv")
df.head()

,Movie Name,Year of Release,Watch Time,Movie Rating,Metascore of movie,Gross,Votes,Description,Ids,TMDB_Id,...,History,Horror,Music,Mystery,Romance,SciFi,TV,Thriller,War,Western
0,The Shawshank Redemption,1994,142,9.3,82.0,28.34,"27,77,378","Over the course of several years, two convicts...","(278, [18, 80])",278.0,...,0,0,0,0,0,0,0,0,0,0
1,The Godfather,1972,175,9.2,100.0,134.97,"19,33,588","Don Vito Corleone, head of a mafia family, dec...","(238, [18, 80])",238.0,...,0,0,0,0,0,0,0,0,0,0
2,The Dark Knight,2008,152,9.0,84.0,534.86,"27,54,087",When the menace known as the Joker wreaks havo...,"(155, [18, 28, 80, 53])",155.0,...,0,0,0,0,0,0,0,1,0,0
3,Schindler's List,1993,195,9.0,95.0,96.9,"13,97,886","In German-occupied Poland during World War II,...","(424, [18, 36, 10752])",424.0,...,1,0,0,0,0,0,0,0,1,0
4,12 Angry Men,1957,96,9.0,97.0,4.36,"8,24,211",The jury in a New York City murder trial is fr...,"(389, [18])",389.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df_dogs = df.loc[df['Description'].str.contains('dog', case=False)]
df_dogs

,Movie Name,Year of Release,Watch Time,Movie Rating,Metascore of movie,Gross,Votes,Description,Ids,TMDB_Id,...,History,Horror,Music,Mystery,Romance,SciFi,TV,Thriller,War,Western
205,Umberto D.,1952,89,8.2,92.0,0.07,"27,087",An elderly man and his dog struggle to survive...,"(833, [18])",833.0,...,0,0,0,0,0,0,0,0,0,0
235,The Wizard of Oz,1939,102,8.1,92.0,2.08,"4,14,083",Young Dorothy Gale and her dog Toto are swept ...,"(630, [12, 14, 10751])",630.0,...,0,0,0,0,0,0,0,0,0,0
263,Hachi: A Dog's Tale,2009,93,8.1,NaN,236,"2,98,283",A college professor bonds with an abandoned do...,"(28178, [18, 10751])",28178.0,...,0,0,0,0,0,0,0,0,0,0
540,Togo,2019,113,7.9,69.0,NaN,"52,579","The story of Togo, the sled dog who led the 19...","(884363, [80, 18])",884363.0,...,0,0,0,0,0,0,0,0,0,0
651,Isle of Dogs,2018,101,7.8,82.0,32.02,"1,81,265","Set in Japan, Isle of Dogs follows a boy's ody...","(399174, [12, 35, 16])",399174.0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df_dogs = df_dogs[df['Movie Name'] != "The Wizard of Oz"]
df_dogs

/var/folders/9s/wjp86gfn3yl56f5kv0hr0_fr0000gn/T/ipykernel_726/176058911.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_dogs = df_dogs[df['Movie Name'] != "The Wizard of Oz"]


,Movie Name,Year of Release,Watch Time,Movie Rating,Metascore of movie,Gross,Votes,Description,Ids,TMDB_Id,...,History,Horror,Music,Mystery,Romance,SciFi,TV,Thriller,War,Western
205,Umberto D.,1952,89,8.2,92.0,0.07,"27,087",An elderly man and his dog struggle to survive...,"(833, [18])",833.0,...,0,0,0,0,0,0,0,0,0,0
263,Hachi: A Dog's Tale,2009,93,8.1,NaN,236,"2,98,283",A college professor bonds with an abandoned do...,"(28178, [18, 10751])",28178.0,...,0,0,0,0,0,0,0,0,0,0
540,Togo,2019,113,7.9,69.0,NaN,"52,579","The story of Togo, the sled dog who led the 19...","(884363, [80, 18])",884363.0,...,0,0,0,0,0,0,0,0,0,0
651,Isle of Dogs,2018,101,7.8,82.0,32.02,"1,81,265","Set in Japan, Isle of Dogs follows a boy's ody...","(399174, [12, 35, 16])",399174.0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df[df['Movie Name'] == 'Up']

,Movie Name,Year of Release,Watch Time,Movie Rating,Metascore of movie,Gross,Votes,Description,Ids,TMDB_Id,...,History,Horror,Music,Mystery,Romance,SciFi,TV,Thriller,War,Western
109,Up,2009,96,8.3,88.0,293,"10,81,937",78-year-old Carl Fredricksen travels to Paradi...,"(14160, [16, 35, 10751, 12])",14160.0,...,0,0,0,0,0,0,0,0,0,0
